In [33]:
import spacy
import coreferee

nlp = spacy.load('en_core_web_trf')
nlp.add_pipe('coreferee')


In [42]:
def resolve_coreference(text):
    doc = nlp(text)
    refined_text = []
    for i in range(len(doc)):
        res = doc._.coref_chains.resolve(doc[i])
        if res is None:
            refined_text += [doc[i]]
        else:
            refined_text += res
    
    return ' '.join([d.text for d in refined_text])


In [47]:
def process_file(input_path, output_path):
    with open(input_path, "r") as f:
        lines = f.readlines()
    
    resolved_lines = []
    for line in lines:
        resolved_lines.append(resolve_coreference(line))

    with open(output_path, 'w') as f:
        for line in resolved_lines:
            f.write(f"{line}")

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed


def process_files(input_files, output_files, num_threads):
    """
    Многопоточная обработка списка входных файлов и сохранение результатов в выходные файлы.
    :param input_files: Список путей к входным файлам.
    :param output_files: Список путей к выходным файлам (должен быть того же размера, что и input_files).
    :param num_threads: Количество потоков для обработки.
    """
    if len(input_files) != len(output_files):
        raise ValueError("Количество входных и выходных файлов должно совпадать.")

    # Используем ThreadPoolExecutor для многопоточной обработки
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [
            executor.submit(process_file, input_file, output_file)
            for input_file, output_file in zip(input_files, output_files)
        ]

        # Ожидаем завершения всех задач
        for future in as_completed(futures):
            try:
                future.result()  # Проверяем на наличие исключений
            except Exception as e:
                print(f"Error processing file: {e}")

/Users/dmitry/Desktop/Projects/coreref_spacy/.venv/lib/python3.11/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [ ]:
import os
from pathlib import Path

input_directory = Path("/home/simon/Desktop/concept-tree/concept-tree/nn_graph_makers/test_files_output")
output_directory = Path("/home/simon/Desktop/concept-tree/concept-tree/nn_graph_makers/test_files_output_anaphora_splitted")